### Step1 - 导入相关包 & 初始化设置

In [ ]:
import os, sys

from langchain.chat_models import init_chat_model
# 常用的消息类型
from langchain.messages import HumanMessage, SystemMessage, ToolMessage, AIMessage, AIMessageChunk

# 默读取当前目录下的 .env 文件, 可以通过 dotenv_path 来修改
from dotenv import load_dotenv
root_dir = os.path.dirname(os.getcwd())
load_dotenv(dotenv_path=os.path.join(root_dir, ".env"))

# 记录日志
from loguru import logger
logger.remove()
logger.add(sys.stderr, level=os.getenv("LOG_LEVEL"))

1

### Step2 - 初始化聊天模型

In [2]:
# LangChain v1 创建聊天模型的方法
chat_model = init_chat_model(
    model_provider="openai",
    # model 也可以写为 <model_provider>:<model_name> 的形式
    # 这样就可以不用指定 model_provider 这个参数了
    model = os.getenv("LMSTUDIO_LLM_MODEL"),
    base_url = os.getenv("LMSTUDIO_BASE_URL"),
    api_key = "LM Studio",
    max_tokens = 1024,

)
chat_model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001E49A57E7B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001E49A57F230>, root_client=<openai.OpenAI object at 0x000001E49A57C2F0>, root_async_client=<openai.AsyncOpenAI object at 0x000001E49A57EF90>, model_name='qwen/qwen3-vl-8b', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://pc-wk1:1234/v1', max_tokens=1024)

### Step3 - 定义聊天模板

In [3]:
messages = [
    SystemMessage(content="你是一个专业的AI助手, 请使用简洁的语言来回答用户的问题."),
]
# 可以用 append 的方式来添加/保存对话信息
messages.append(HumanMessage(content="请简单介绍一下你自己."))
messages

[SystemMessage(content='你是一个专业的AI助手, 请使用简洁的语言来回答用户的问题.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='请简单介绍一下你自己.', additional_kwargs={}, response_metadata={})]

### Step4 - 执行对话

In [4]:
# 方法1: invoke 非流式输出
respone = chat_model.invoke(messages)

# 类型为: <class 'langchain_core.messages.ai.AIMessage'>
logger.debug(type(respone))

for msg in respone:
    print(msg)

2025-12-04 23:14:09.084 | DEBUG    | __main__:<module>:5 - <class 'langchain_core.messages.ai.AIMessage'>


('content', '我是一个AI助手，可以回答问题、提供信息、帮助写作、进行逻辑推理等。我的目标是为你提供准确、有用的信息，帮助你解决问题或完成任务。如果你有任何问题，欢迎随时问我！')
('additional_kwargs', {'refusal': None})
('response_metadata', {'token_usage': {'completion_tokens': 46, 'prompt_tokens': 34, 'total_tokens': 80, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen/qwen3-vl-8b', 'system_fingerprint': 'qwen/qwen3-vl-8b', 'id': 'chatcmpl-ugo0qi1fdhhowj7fg3w31s', 'finish_reason': 'stop', 'logprobs': None})
('type', 'ai')
('name', None)
('id', 'lc_run--76dd35f6-f1bf-428b-b0dd-734ceba8c5c6-0')
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 34, 'output_tokens': 46, 'total_tokens': 80, 'input_token_details': {}, 'output_token_details': {}})


In [5]:
# 方法2: steam 流式输出
display_debug = False

for chunk in chat_model.stream(messages):
    # 判断内容是否是 AIMessageChunk, 如果是则打印内容
    if isinstance(chunk, AIMessageChunk):
        print(chunk.content, end="", flush=True)
    # 展示其中一个 chunk 中的内容
    if display_debug is False:
        logger.debug(f"chunk type: {type(chunk)}")
        for i in chunk:
            print(i)
        display_debug = True



我是

2025-12-04 23:14:09.199 | DEBUG    | __main__:<module>:10 - chunk type: <class 'langchain_core.messages.ai.AIMessageChunk'>


('content', '我是')
('additional_kwargs', {})
('response_metadata', {'model_provider': 'openai'})
('type', 'AIMessageChunk')
('name', None)
('id', 'lc_run--ffae9783-2271-47e4-a0fc-5602a8f60c86')
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', None)
('tool_call_chunks', [])
('chunk_position', None)
Qwen，是阿里云研发的一款超大规模语言模型。我可以回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等，还能表达观点，玩游戏等。我的目标是成为你可靠且贴心的AI助手。有什么我可以帮你的吗？